In [ ]:
# CONFIG
ICL_model_name = "tiiuae/falcon-7b" # note: "meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b", "meta-llama/Meta-Llama-3-70B", "tiiuae/falcon-40b"
quantized = False if ICL_model_name in ["meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b"] else True
encoder_model_name = "BAAI/bge-m3"
token = 'your token here'
ICL_selected_token_type = "label_words" # "none" (natural ICL sample end), "label_words", "last_sentence_token"
k = 4 # 0, 1, 2, 4, 8, 12
dataset_index = 2 # 0, 1, 2, 3, 4, 5
pes_dataset_index = 7

In [2]:
import sys
sys.path.append("ICL_Inference_Dynamics_Released")
from util import load_model_and_data, kernel_alignment, inference
import StaICC

bench = StaICC.Normal(k)
prompts, queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[dataset_index], ICL_selected_token_type)
_, pesudo_queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[pes_dataset_index])

Loading data...

1 in 10 Data loaded:  GLUE-SST2 

2 in 10 Data loaded:  rotten_tomatoes 

3 in 10 Data loaded:  financial_phrasebank 

4 in 10 Data loaded:  SST5 

5 in 10 Data loaded:  TREC 

6 in 10 Data loaded:  AGNews 

7 in 10 Data loaded:  Subjective 

8 in 10 Data loaded:  tweet_eval_emotion 

9 in 10 Data loaded:  tweet_eval_hate 

10 in 10 Data loaded:  hate_speech_18 

Data loaded successfully.

Initializing experimentor on k = 4...

Ready.



In [3]:
vars_dict = vars() if "ICL_model" in vars() else locals()
if "ICL_model" not in vars_dict:
    ICL_model, ICL_tknz = load_model_and_data.load_ICL_model(ICL_model_name, huggingface_token = token, quantized = quantized)
    encoder_model, encoder_tknz = load_model_and_data.load_encode_model(encoder_model_name, huggingface_token = token)
    loaded = True

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/s2320415/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
ICL_hidden_states = inference.ICL_inference_to_hidden_states(ICL_model, ICL_tknz, prompts)
encoder_feature = inference.encoder_inference_to_feature(encoder_model, encoder_tknz, queries)
pesudo_encoder_feature = inference.encoder_inference_to_feature(encoder_model, encoder_tknz, pesudo_queries)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:06<00:00, 80.82it/s]


In [5]:
ICL_sim_graph = []
for layer_hidden_state in ICL_hidden_states:
    ICL_sim_graph.append(kernel_alignment.sim_graph(layer_hidden_state))
encoder_sim_graph = kernel_alignment.sim_graph(encoder_feature)
pesudo_encoder_sim_graph = kernel_alignment.sim_graph(pesudo_encoder_feature)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:01<00:00, 322.71it/s]


In [6]:
res = []
pesudo_res = []
for layer_sim_graph in ICL_sim_graph:
    res.append(kernel_alignment.kernel_alignment(layer_sim_graph, encoder_sim_graph))
    pesudo_res.append(kernel_alignment.kernel_alignment(layer_sim_graph, pesudo_encoder_sim_graph))

In [7]:
for line in res:
    print(line[0])

0.21746826171875
0.29425048828125
0.281158447265625
0.289215087890625
0.29742431640625
0.29510498046875
0.26312255859375
0.268585205078125
0.258697509765625
0.257720947265625
0.25604248046875
0.254119873046875
0.25616455078125
0.2593994140625
0.2598876953125
0.26123046875
0.260040283203125
0.25762939453125
0.256683349609375
0.25164794921875
0.2479248046875
0.2430419921875
0.240814208984375
0.237762451171875
0.23712158203125
0.237518310546875
0.23590087890625
0.23516845703125
0.234130859375
0.231048583984375
0.229949951171875
0.23101806640625
0.22137451171875


In [8]:
# Result file organization:
# (
#    res: layer_number * (alignment mean, alignment std, alignment of invidivual sample),
#    pesudo_res: layer_number * (alignment mean, alignment std, alignment of invidivual sample),
# )

'''
dataset index:

1 in 10 Data loaded:  GLUE-SST2 

2 in 10 Data loaded:  rotten_tomatoes 

3 in 10 Data loaded:  financial_phrasebank 

4 in 10 Data loaded:  SST5 

5 in 10 Data loaded:  TREC 

6 in 10 Data loaded:  AGNews 

7 in 10 Data loaded:  Subjective 

8 in 10 Data loaded:  tweet_eval_emotion 

9 in 10 Data loaded:  tweet_eval_hate 

10 in 10 Data loaded:  hate_speech_18 
'''

import pickle

data_file_name = "../data/" + ICL_model_name.replace('/', '_')+ "," + encoder_model_name.replace('/', '_')+ "," + ICL_selected_token_type + "," + str(k) + "," + str(dataset_index + 1) + "," + str(pes_dataset_index + 1) + ".pickle"
with open(data_file_name, 'wb') as f:
    pickle.dump((res, pesudo_res), f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/tiiuae_falcon-7b,BAAI_bge-m3,label_words,4,3,8.pickle'